In [1]:
!pwd

/home/jovyan/mathieu/gent-commuters/Thomas/Notebooks


In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")
sys.path.append("../../../../Thomas/gent-commuters/Thomas/Notebooks/")
from utils import *

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
track = "track6"
from track6 import *

In [4]:
backbone = generate_backbone(bpm,beat_per_bar,bar_per_emphasis, n_bars_buildup, progression)

handling {'part': 'intro', 'buildup': 1.0, 'duration': 4, 'tracks': [('base', 'no'), ('chords', 'no'), ('drums1', 'no'), ('drums2', 'no'), ('melody1', 'yes'), ('melody2', 'no')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 12, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'no'), ('melody1', 'yes'), ('melody2', 'no')]}
handling {'part': 'pause', 'buildup': 1.0, 'duration': 4, 'tracks': [('base', 'yes'), ('chords', 'no'), ('drums1', 'no'), ('drums2', 'no'), ('melody1', 'no'), ('melody2', 'no')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 12, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'yes'), ('melody1', 'yes'), ('melody2', 'yes')]}
handling {'part': 'verse', 'buildup': 0.5, 'duration': 6, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'yes'), ('melody1', 'yes'), ('melody2', 'yes')]}
handling {'part': 'verse', 'buildup': 0.0, 'duration': 6, 'tracks': [('base', 'yes'), ('ch

In [5]:
from magenta.music.midi_io import sequence_proto_to_midi_file, sequence_proto_to_pretty_midi
import pickle

with open("pickled_"+track+"_drums", "rb") as f:
    full_drum_seq=pickle.load(file=f)

# Control change

In [6]:
for cc in [93,91,75,5]:
    chorus_target = 128

    start_chorus = 0
    drums = full_drum_seq.instruments
    for b in backbone:
        buildup = b["buildup_factor"]

        start_time = b["start_time"]
        end_time = b["end_time"]
        #
        # Chorus
        #
        if buildup > 0:
            end_chorus = math.floor(buildup**2 * chorus_target)

            for chorus in range(start_chorus, end_chorus):

                rat = (chorus - start_chorus)/(end_chorus - start_chorus)

                time = start_time * (1-rat) + end_time * rat

                for drum in drums:
                    drum.control_changes.append(pretty_midi.ControlChange(cc, chorus, time))

            start_chorus = end_chorus+1
        elif start_chorus > 0:
            for drum in drums:
                drum.control_changes.append(pretty_midi.ControlChange(cc, 0, start_time))
            start_chorus = 0

# PitchBend

In [7]:
pitch_target = 8191

start_pitch = 0
drums = full_drum_seq.instruments

for drum in drums:
    drum.control_changes.append(pretty_midi.ControlChange(100, 0, 0))
    drum.control_changes.append(pretty_midi.ControlChange(101, 0, 0))
    drum.control_changes.append(pretty_midi.ControlChange(6, 64, 1))
    
for b in backbone:
    buildup = b["buildup_factor"]
    
    start_time = b["start_time"]
    end_time = b["end_time"]
    #
    # Pitch
    #
    if buildup > 0:
        end_pitch = math.floor(buildup**2 * pitch_target)

        for pitch in range(start_pitch, end_pitch):

            rat = (pitch - start_pitch)/(end_pitch - start_pitch)

            time = start_time * (1-rat) + end_time * rat

            for drum in drums:
                drum.pitch_bends.append(pretty_midi.PitchBend(pitch, time))
            
        start_pitch = end_pitch+1
    elif start_pitch > 0:
        for drum in drums:
            drum.pitch_bends.append(pretty_midi.PitchBend(0, start_time))
        start_pitch = 0

    

In [14]:
generate_mp3(full_drum_seq.fluidsynth(),ext="_"+track)

In [8]:
full_drum_seq.write('drums_'+track+'.mid')


# Merge chords and melody

In [9]:
melody_filename = os.path.join('../../components',root_name+'_melody.mid')
components_folder = os.path.join('../../components',root_name)
os.makedirs(components_folder, exist_ok=True)

In [10]:
chords = pretty_midi.PrettyMIDI(os.path.join(components_folder,'chords_16bar.mid')).instruments[0]
bass = pretty_midi.PrettyMIDI(os.path.join(components_folder,'bass_16bar.mid')).instruments[0]
melody = pretty_midi.PrettyMIDI(os.path.join(components_folder,'melody_16bar.mid')).instruments[0]

In [11]:
bar_duration = beat_per_bar*60.0/bpm

chords_shifted = chords.notes
acc=[]
for n in chords_shifted:
    start=n.start+16*bar_duration
    end=n.end+16*bar_duration
    pitch=n.pitch
    vel = n.velocity
    acc.append(pretty_midi.Note(vel,pitch,start,end))

for n in acc:
    chords.notes.append(n)
    
bass_shift = [n for n in bass.notes]
for n in bass_shift:
    n.start=n.start+16*bar_duration
    n.end=n.end+16*bar_duration
bass.notes=bass_shift

melody_shift = [n for n in melody.notes]
i=0
for n in melody_shift:
    if i%7 !=0:
        n.vel = 0
    else:
        n.vel = 127
    n.start=n.start+16*bar_duration
    n.end=n.end+16*bar_duration
    i+=1
melody.notes=melody_shift

In [12]:
full_track = full_drum_seq
full_track.instruments.append(chords)
full_track.instruments.append(bass)
full_track.instruments.append(melody)

In [15]:
generate_mp3(full_track.fluidsynth(),ext="_"+track)

In [13]:
full_track.write('full_'+track+'.mid')


In [14]:
track

'track6'